In [2]:
text = "Myself Akshat Rai Laddha, working as Machine Learning engineer at Carelon Global Solutions in Bangalore. Proficient in public speaking and coding"

In [5]:
# Loading the pipeline from hub
from transformers import pipeline
# Pipeline handles the preprocessing and post processing steps
model_checkpoint_m1 = "balamurugan1603/bert-finetuned-ner"
namedEntityRecogniser_m1 = pipeline(
    "token-classification", model=model_checkpoint_m1, aggregation_strategy="simple"
)

In [7]:
m1_results=namedEntityRecogniser_m1([text])
print(m1_results)

[[{'entity_group': 'PER', 'score': 0.99715745, 'word': 'Akshat Rai Laddha', 'start': 7, 'end': 24}, {'entity_group': 'ORG', 'score': 0.99314034, 'word': 'Carelon Global Solutions', 'start': 66, 'end': 90}, {'entity_group': 'LOC', 'score': 0.9948565, 'word': 'Bangalore', 'start': 94, 'end': 103}]]


In [8]:
# Model 2: for skill recognition
model_checkpoint_m2 = "algiraldohe/lm-ner-linkedin-skills-recognition"
namedEntityRecogniser_m2 = pipeline(
    "token-classification", model=model_checkpoint_m2, aggregation_strategy="simple"
)

In [9]:
m2_results = namedEntityRecogniser_m2([text])
print(m2_results)

[[{'entity_group': 'TECHNICAL', 'score': 0.99462366, 'word': 'machine learning', 'start': 37, 'end': 53}, {'entity_group': 'SOFT', 'score': 0.7944664, 'word': 'public speaking', 'start': 119, 'end': 134}]]


extracted results from both the models.

### Merging results

In [10]:
m1_results.extend(m2_results)

In [13]:
# Merge the two lists into a single list
merged_results = m1_results[0]
for entity in m1_results[1]:
    existing_entity = next((ent for ent in merged_results if ent['start'] == entity['start']), None)
    if existing_entity is None or entity['score'] > existing_entity['score']:
        merged_results.append(entity)

# Sort the merged list based on 'start' index in ascending order
sorted_results= sorted(merged_results, key=lambda x: x['start'])

In [14]:
print(sorted_results)

[{'entity_group': 'PER', 'score': 0.99715745, 'word': 'Akshat Rai Laddha', 'start': 7, 'end': 24}, {'entity_group': 'TECHNICAL', 'score': 0.99462366, 'word': 'machine learning', 'start': 37, 'end': 53}, {'entity_group': 'ORG', 'score': 0.99314034, 'word': 'Carelon Global Solutions', 'start': 66, 'end': 90}, {'entity_group': 'LOC', 'score': 0.9948565, 'word': 'Bangalore', 'start': 94, 'end': 103}, {'entity_group': 'SOFT', 'score': 0.7944664, 'word': 'public speaking', 'start': 119, 'end': 134}]


In [22]:
final_result=[sorted_results]

### Visualization

In [23]:
from spacy import displacy

In [24]:
def visualize(pipeline_output, texts):
    for i in range(len(final_result)):
        entities = []
        for ents in final_result[i]:
            entities.append({"end": ents["end"], "label": ents["entity_group"], "start": ents["start"]})
        displacy.render({
            "ents": entities,
            "text": texts[i]
        }, style="ent", manual=True)
visualize(final_result,[text])

### Successfully extracted entities with different labels such as Person, Organization, Location, Skills , Miscellaneous etc.